# M2608.001300 기계학습 기초 및 전기정보 응용<br> Assignment 4: Hidden Markov Model (HMM)

In [1]:
# Code from Chapter 16 of Machine Learning: An Algorithmic Perspective (2nd Edition)
# by Stephen Marsland (http://stephenmonika.net)

# You are free to use, change, or redistribute the code in any way you wish for
# non-commercial purposes, but please maintain the name of the original author.
# This code comes with no warranty of any kind.

# Stephen Marsland, 2008, 2014

# A basic Hidden Markov Model

## Setup
Check that Python 3.5 or later is installed (although Python 2.x may work, it is deprecated so we strongly recommend you use Python 3 instead).

In [2]:
# Python >=3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Common imports
import numpy as np
from pandas import DataFrame

# to make this notebook's output stable across runs
np.random.seed(4)

## Problem 1. Evaluation problem
Get the probability of seeing an observation. <br>
We can use the following 2 algorithms:
<br>
* <font color=blue>*Forward algorithm*</font> (dynamic programming)
* <font color=blue>*Backward algorithm*</font> (dynamic programming)

In [3]:
scaling = False

In [4]:
"""
Let's denote the parameters of HMM model as,

pi : initial state probability
a : transition probability
b : emission probability
obs : observation sequence
"""
def HMMfwd(pi, a, b, obs):
    # TODO: Forward algorithm
    ####################### YOUR CODE HERE #######################
    nStates = np.shape(b)[0]
    T = np.shape(obs)[0]

    alpha = np.zeros((nStates,T))

    alpha[:,0] = pi*b[:,obs[0]]

    for t in range(1,T):
        for s in range(nStates):
            alpha[s,t] = b[s,obs[t]] * np.sum(alpha[:,t-1] * a[:,s])

    c = np.ones((T))
    if scaling:
        for t in range(T):
            c[t] = np.sum(alpha[:,t])
            alpha[:,t] /= c[t]
    ##############################################################
    return alpha, c

def HMMbwd(a, b, obs, c):
    # TODO: Backward algorithm
    ####################### YOUR CODE HERE #######################
    nStates = np.shape(b)[0]
    T = np.shape(obs)[0]

    beta = np.zeros((nStates,T))

    beta[:,T-1] = 1.0 #aLast

    for t in range(T-2,-1,-1):
        for s in range(nStates):
            beta[s,t] = np.sum(b[:,obs[t+1]] * beta[:,t+1] * a[s,:])

    for t in range(T):
        beta[:,t] /= c[t]
    #beta[:,0] = b[:,obs[0]] * np.sum(beta[:,1] * pi)
    ##############################################################
    return beta

## Problem 2. Decoding problem
Get the underlying state sequence of an observation. <br>
We can use the <font color=blue>*Viterbi algorithm*</font> (dynamic programming).

In [5]:
def Viterbi(pi, a, b, obs):
    # TODO: Viterbi algorithm
    ####################### YOUR CODE HERE #######################
    nStates = np.shape(b)[0]
    T = np.shape(obs)[0]

    path = np.zeros(T, dtype=int)
    delta = np.zeros((nStates,T))
    phi = np.zeros((nStates,T), dtype=int)

    delta[:,0] = pi * b[:,obs[0]]
    phi[:,0] = 0

    for t in range(1,T):
        for s in range(nStates):
            delta[s,t] = np.max(delta[:,t-1]*a[:,s])*b[s,obs[t]]
            phi[s,t] = np.argmax(delta[:,t-1]*a[:,s])

    path[T-1] = np.argmax(delta[:,T-1])
    for t in range(T-2,-1,-1):
        path[t] = phi[path[t+1],t+1]
    ##############################################################
    return path, delta, phi

## Problem 3. Learning problem
Get the best parameters for the model. <br>
We can use the following 3 algorithms:
* *MLE* (maximum likelihood estimation)
* *Viterbi training* (different from Viterbi decoding)
* *Baum Welch algorithm* (a.k.a. forward-backward algorithm, uses expectation-maximization)

Here we implement the <font color=blue>*Baum Welch algorithm*</font>.

In [17]:
def BaumWelch(obs, nStates):

    T = np.shape(obs)[0]
    xi = np.zeros((nStates,nStates,T))

    # Initialise pi, a, b randomly
    pi = 1./nStates*np.ones((nStates))
    a = np.random.rand(nStates,nStates)
    b = np.random.rand(nStates,np.max(obs)+1)

    tol = 1e-5
    error = tol+1
    maxits = 100
    nits = 0

    # TODO: Baum Welch algorithm
    ####################### YOUR CODE HERE #######################
    while ((error > tol) & (nits < maxits)):
        nits += 1
        oldpi = pi.copy()
        olda = a.copy()
        oldb = b.copy()

        # E step
        alpha,c = HMMfwd(pi, a, b, obs)
        beta = HMMbwd(a, b, obs, c) 

        for t in range(T-1):
            for i in range(nStates):
                for j in range(nStates):
                    xi[i,j,t] = alpha[i,t]*a[i,j]*b[j,obs[t+1]]*beta[j,t+1]
            xi[:,:,t] /= np.sum(xi[:,:,t])

        # The last step has no b, beta in
        for i in range(nStates):
            for j in range(nStates):
                xi[i,j,T-1] = alpha[i,T-1]*a[i,j]
        xi[:,:,T-1] /= np.sum(xi[:,:,T-1])

        # M step
        for i in range(nStates):
            pi[i] = np.sum(xi[i,:,0])
            for j in range(nStates):
                a[i,j] = np.sum(xi[i,j,:T-1])/np.sum(xi[i,:,:T-1])

            for k in range(max(obs)):
                found = (obs==k).nonzero()
                b[i,k] = np.sum(xi[i,:,found])/np.sum(xi[i,:,:])

        error = (np.abs(a-olda)).max() + (np.abs(b-oldb)).max() 
        print(nits, error, 1./np.sum(1./c), np.sum(alpha[:,T-1]))
    ##############################################################
    return pi, a, b

## Toy examples
Here are the parameters for HMM model.

In [7]:
transition_data = {'state': ['TV', 'Bar', 'Party', 'Study'],
                          'TV'    : [0.4, 0.3, 0.1, 0.2],
                          'Bar'   : [0.6, 0.05, 0.1, 0.25],
                          'Party' : [0.7, 0.05, 0.05, 0.2],
                          'Study' : [0.3, 0.4, 0.25, 0.05]}

emission_data = {'observation': ['tired', 'hangover', 'anxiety', 'good'],
                        'TV'    : [0.2, 0.1, 0.2, 0.5],
                        'Bar'   : [0.4, 0.2, 0.1, 0.3],
                        'Party' : [0.3, 0.4, 0.2, 0.1],
                        'Study' : [0.3, 0.05, 0.3, 0.35]}

transition_probability = DataFrame(transition_data, columns=['state', 'TV', 'Bar', 'Party', 'Study'])
emission_probability = DataFrame(emission_data, columns=['observation', 'TV', 'Bar', 'Party', 'Study'])

In [8]:
transition_probability

,state,TV,Bar,Party,Study
0,TV,0.4,0.60,0.70,0.30
1,Bar,0.3,0.05,0.05,0.40
2,Party,0.1,0.10,0.05,0.25
3,Study,0.2,0.25,0.20,0.05


In [9]:
emission_probability

,observation,TV,Bar,Party,Study
0,tired,0.2,0.4,0.3,0.30
1,hangover,0.1,0.2,0.4,0.05
2,anxiety,0.2,0.1,0.2,0.30
3,good,0.5,0.3,0.1,0.35


In [10]:
try:
    a = transition_probability.drop(['state'], axis=1).to_numpy()
    b = emission_probability.drop(['observation'], axis=1).to_numpy()
except:
    a = transition_probability.drop(['state'], axis=1).values
    b = emission_probability.drop(['observation'], axis=1).values

In [11]:
# Assume the initial state probabilites are all equal to 0.25
pi = np.array([0.25, 0.25, 0.25, 0.25])
# obs = np.array(['tired', 'tired', 'good', 'hangover', 'hangover', 'anxiety', 'hangover', 'good'])
obs = np.array([0, 0, 3, 1, 1, 2, 1, 3])

nStates = np.shape(b)[0]
T = np.shape(obs)[0]

## Problem 4-1. 
What is the probability of seeing current observation?

In [12]:
####################### YOUR CODE HERE #######################
alpha, c = HMMfwd(pi, a, b, obs)
print('=== Probability of the Current Observation ===\n', np.sum(alpha[:,-1]))
##############################################################

=== Probability of the Current Observation ===
 2.1380223436320317e-05


## Problem 4-2.
Given the current observation, what is the most likely sequence of states?

In [13]:
####################### YOUR CODE HERE #######################
path, delta, phi = Viterbi(pi, a, b, obs)
print('=== Most Likely Sequence ===\n', path)
print('====== Path Reference ======\n', phi)
##############################################################

=== Most Likely Sequence ===
 [3 0 0 0 0 1 0 2]
====== Path Reference ======
 [[0 3 0 0 0 0 1 0]
 [0 3 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 2 2 0 0 1 0]]


## Problem 4-3.
Given the current observation and the model, find the model parameters (transition probability, emission probability, initial state probability) that best fits the data.

In [18]:
####################### YOUR CODE HERE #######################
pi, a, b = BaumWelch(obs, nStates)
print('===== Transition Probability ====\n', a)
print('====== Emission Probability =====\n', b)
print('=== Initial State Probability ===\n', pi)
##############################################################

1 1.2553719997443726 0.125 0.6428743098946529
2 0.1413069220135454 0.125 9.155052754146224e-05
3 0.129292832750675 0.125 0.00010103391588834797
4 0.16104877463843692 0.125 0.0001123413753110763
5 0.18836878278163527 0.125 0.0001317606426238109
6 0.22557664480296424 0.125 0.0001724120741806941
7 0.25196084447873107 0.125 0.00027436029675387996
8 0.3618165058983672 0.125 0.0006262616995234997
9 0.31909663012011963 0.125 0.0021149800424338137
10 0.19928471849548807 0.125 0.005530599699368257
11 0.18406137452980043 0.125 0.00894790959168759
12 0.11363555021236155 0.125 0.012021288898433744
13 0.06531816893968478 0.125 0.013115605684131315
14 0.041041473019478525 0.125 0.012860143770728244
15 0.028139182081453273 0.125 0.012534705196424049
16 0.020126245832081502 0.125 0.012294515158403569
17 0.014759576324243184 0.125 0.012120653823612467
18 0.011010855081655557 0.125 0.011992998130418068
19 0.008317553761690577 0.125 0.011897831164375369
20 0.006342183838533133 0.125 0.01182600802251132
2